In [2]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max.columns', 100)
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from calendar import monthrange 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
import wikipedia
import random
import requests

In [3]:
pd.set_option('display.width', 500) 
pd.set_option ('display.max_columns' , 500)
pd.set_option("display.max_rows",500)


In [4]:
def find_the_nearest_corr (cor, list_cor):
    def num_same_sighn (num, list_of_numbers):
        def sighn_num (num):
            if num < 0:
                return -1
            elif num == 0:
                return 0
            elif num > 0:
                return 1
        sighn_num_ = sighn_num(num)
        sighns_list_num = [*map(lambda x: sighn_num(x)==sighn_num_, list_of_numbers)]
        return sighns_list_num
    numerate_cors = enumerate(list_cor)
    a = cor[0]
    b = cor[1]
    a_cors = map(lambda x: x[0], list_cor)
    b_cors = map(lambda x: x[1], list_cor)
    same_sighn_a = num_same_sighn(a, a_cors)
    same_sighn_b = num_same_sighn(b, b_cors)
    bool_tuples = zip(same_sighn_a, same_sighn_b)
    equal_tuples_bool = [*map(all, bool_tuples)]
    num_equal_bools = []
    pos = 0
    for bool_ in equal_tuples_bool:
        if bool_:
            num_equal_bools.append(pos)
        pos += 1
    same_sighn_cor = np.array(list_cor)[num_equal_bools]
    Euclidean_distances = []
    for a_ , b_ in same_sighn_cor:
        dist = np.sqrt((a - a_)**2 + (b - b_)**2)
        Euclidean_distances.append(dist)
    the_nearest_cor = same_sighn_cor[np.argmin(Euclidean_distances)]
    return tuple(the_nearest_cor) 

In [6]:
New_xtrain = pd.read_csv('Data for comp/Extended all data.csv')

In [7]:
response = requests.get('https://climatecharts.net/weatherstations-api/getAllStations')

In [8]:
New_air_ports_data = pd.read_csv('Data for comp/Extended Air Port Data.csv')

In [9]:
data_stations = pd.DataFrame(response.json())[['id','country', 'name', 'max_year', 'lng','lat']]

In [10]:
data_stations['Point'] = (data_stations['lat'].apply(str) + ',' + data_stations['lng'].apply(str)).apply(lambda x: (float(x.split(',')[0]),float(x.split(',')[1])))

In [11]:
data_stations

,id,country,name,max_year,lng,lat,Point
0,30984721000,Peru,San Juan,1997,-75.169998,-15.38,"(-15.380000114440918, -75.16999816894531)"
1,30984735000,Peru,Juliaca,2019,-70.150002,-15.48,"(-15.479999542236328, -70.1500015258789)"
2,50194703000,Australia,Bourke,1992,145.929993,-30.08,"(-30.079999923706055, 145.92999267578125)"
3,51291541000,Solomon Islands,Santa Cruz,1986,165.800003,-10.70,"(-10.699999809265137, 165.8000030517578)"
4,51791780000,Tonga,Vavau,1999,-173.979996,-18.65,"(-18.649999618530273, -173.97999572753906)"
...,...,...,...,...,...,...,...
4330,63826179000,Russian Federation (European Sector),Novgorod,2019,31.250000,58.52,"(58.52000045776367, 31.25)"
4331,63826258000,Russian Federation (European Sector),Pskov,2019,28.420000,57.82,"(57.81999969482422, 28.420000076293945)"
4332,63826781000,Russian Federation (European Sector),Smolensk,2019,32.070000,54.75,"(54.75, 32.06999969482422)"
4333,64308227001,Spain,Torrejon,1967,-3.450000,40.48,"(40.47999954223633, -3.450000047683716)"


In [12]:
All_point = data_stations['Point'].tolist()
for i in np.arange(New_air_ports_data.shape[0]):
    point = (New_air_ports_data.loc[i, 'Latitude'], New_air_ports_data.loc[i, 'Longitude'])
    the_nearest_point = find_the_nearest_corr(point, All_point)
    pos = All_point.index(the_nearest_point)
    Station_id = data_stations.loc[pos, 'id']
    max_year = data_stations.loc[pos, 'max_year']
    New_air_ports_data.loc[i, 'Station ID'] = Station_id
    New_air_ports_data.loc[i, 'max_year'] = max_year
    

In [13]:
New_air_ports_data.loc[2, 'Station ID']

42572323001.0

In [14]:
New_air_ports_data

,Unnamed: 0,Airport,City,State,IATA,Latitude,Longitude,Acre,Station ID,max_year
0,0,Birmingham–Shuttlesworth International Airport,Birmingham,Alabama,BHM,33.563900,-86.752200,2170.0,4.257223e+10,2019.0
1,1,Dothan Regional Airport,Dothan,Alabama,DHN,31.321100,-85.449400,1150.0,4.257478e+10,1969.0
2,2,Huntsville International Airport,Huntsville,Alabama,HSV,34.637200,-86.775000,6000.0,4.257232e+10,2004.0
3,3,Mobile Regional Airport,Mobile,Alabama,MOB,30.691400,-88.242800,1717.0,4.257222e+10,2019.0
4,4,Montgomery Regional Airport,Montgomery,Alabama,MGM,32.300600,-86.393900,1907.0,4.257223e+10,2019.0
5,5,Adak Airport,Adak,Alaska,ADK,51.878100,-176.646100,2384.0,4.257045e+10,1996.0
6,6,Lake Hood Seaplane Base,Anchorage,Alaska,NaN,61.216700,-149.900000,2384.0,4.257027e+10,1970.0
7,7,Merrill Field,Anchorage,Alaska,MRI,61.213600,-149.844400,436.0,4.257027e+10,1970.0
8,8,Ted Stevens Anchorage International Airport,Anchorage,Alaska,ANC,61.174200,-149.998300,4608.0,4.257027e+10,2019.0
9,9,Aniak Airport,Aniak,Alaska,ANI,61.581700,-159.543100,1722.0,4.257023e+10,1964.0


In [15]:
New_air_ports_data.loc[0, 'Station ID']

42572228000.0

In [16]:
response_climate = requests.get('https://climatecharts.net/weatherstations-api/getStationData?stationId=42572228000&maxYear=2019')

In [17]:
pd.DataFrame(response_climate.json())

,numYears,temp,minYear,maxYear,prec
0,2019,"{'median': 5.99, 'month': 1, 'numGaps': 1953, ...",0,2019,"{'median': 130.4, 'month': 1, 'numGaps': 1965,..."
1,2019,"{'median': 8.74, 'month': 2, 'numGaps': 1953, ...",0,2019,"{'median': 110, 'month': 2, 'numGaps': 1965, '..."
2,2019,"{'median': 12.69, 'month': 3, 'numGaps': 1954,...",0,2019,"{'median': 141.7, 'month': 3, 'numGaps': 1965,..."
3,2019,"{'median': 17.29, 'month': 4, 'numGaps': 1954,...",0,2019,"{'median': 109, 'month': 4, 'numGaps': 1965, '..."
4,2019,"{'median': 21.7, 'month': 5, 'numGaps': 1954, ...",0,2019,"{'median': 98.4, 'month': 5, 'numGaps': 1965, ..."
5,2019,"{'median': 25.31, 'month': 6, 'numGaps': 1954,...",0,2019,"{'median': 95.15, 'month': 6, 'numGaps': 1965,..."
6,2019,"{'median': 27.09, 'month': 7, 'numGaps': 1954,...",0,2019,"{'median': 119.35, 'month': 7, 'numGaps': 1965..."
7,2019,"{'median': 26.5, 'month': 8, 'numGaps': 1956, ...",0,2019,"{'median': 88.6, 'month': 8, 'numGaps': 1966, ..."
8,2019,"{'median': 23.525002, 'month': 9, 'numGaps': 1...",0,2019,"{'median': 83.95, 'month': 9, 'numGaps': 1967,..."
9,2019,"{'median': 17.7, 'month': 10, 'numGaps': 1955,...",0,2019,"{'median': 63.8, 'month': 10, 'numGaps': 1966,..."


In [18]:
my_dict1 = dict()
for i in pd.DataFrame(response_climate.json())['prec']:
    print((i['month'],i['mean']))

(1, 130.3611)
(2, 121.05926)
(3, 152.32034)
(4, 118.74815)
(5, 118.231476)
(6, 103.00925)
(7, 131.1)
(8, 100.19624)
(9, 102.70961)
(10, 69.08868)
(11, 107.87308)
(12, 120.63269)


In [19]:
Airport_temp = dict()
Airport_prec = dict()
AITA = New_air_ports_data['IATA'].tolist()
Station_id = New_air_ports_data['Station ID'].tolist()
Max_year = New_air_ports_data['max_year'].tolist()
AITA_Id_year = [*zip(AITA, Station_id, Max_year)]
for AITA , ID, year in AITA_Id_year:
    ID = int(ID)
    year = int(year)
    response = requests.get('https://climatecharts.net/weatherstations-api/getStationData?stationId='+str(ID)+'&maxYear='+str(year))
    month_temp = dict()
    for dict_ in pd.DataFrame(response.json())['temp']:
        month_temp[dict_['month']] = dict_['mean']
    month_prec = dict()
    for dict__ in pd.DataFrame(response.json())['prec']:
        month_prec[dict__['month']] = dict__['mean']
    Airport_temp[AITA] = month_temp
    Airport_prec[AITA] = month_prec

In [20]:
New_xtrain['Month Origin Temp'] = np.zeros(New_xtrain.shape[0])
New_xtrain['Month Origin Prec'] = np.zeros(New_xtrain.shape[0])
New_xtrain['Month Dest Temp'] = np.zeros(New_xtrain.shape[0])
New_xtrain['Month Dest Prec'] = np.zeros(New_xtrain.shape[0])

In [22]:
for i in np.arange(New_xtrain.shape[0]):
    try:
        Origin_AITA = New_xtrain.loc[i,'Origin']
        Dest_AITA = New_xtrain.loc[i,'Dest']
        Month = New_xtrain.loc[i,'Month']
        New_xtrain.loc[i, 'Month Origin Temp'] = Airport_temp[Origin_AITA][Month]
        New_xtrain.loc[i, 'Month Origin Prec'] = Airport_prec[Origin_AITA][Month]
        New_xtrain.loc[i,'Month Dest Temp'] = Airport_temp[Dest_AITA][Month]
        New_xtrain.loc[i, 'Month Dest Prec'] = Airport_prec[Dest_AITA][Month]
    except:
        New_xtrain.loc[i, 'Month Origin Temp'] = np.NaN
        New_xtrain.loc[i, 'Month Origin Prec'] = np.NaN
        New_xtrain.loc[i,'Month Dest Temp'] = np.NaN
        New_xtrain.loc[i, 'Month Dest Prec'] = np.NaN

In [27]:
New_xtrain.to_csv('Exended all data climate.csv')